In [16]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

# Checking Total Derivatives

If you want to check the analytic derivatives of your model (or just part of it) against finite difference or complex-step approximations, you can use `check_totals`(). You should always converge your model before calling this method. By default, this method checks the derivatives of all of the driver responses (objectives, constraints) with respect to the des_vars, though you can also specify the variables you want to check. Derivatives are computed and compared in an unscaled form by default, but you can optionally request for them to be computed in scaled form using the `ref` and `ref0` that were declared when adding the constraints, objectives, and des_vars.

```{Note}
You should probably **not** use this method until you’ve used `check_partials`() to verify the partials for each component in your model. `check_totals`() is a blunt instrument, since it can only tell you that there is a problem, but will not give you much insight into which component or group is causing the problem.

``` 

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.check_totals
        :noindex:
```

Please check out the :ref:`Complex Step Guidelines <approximating_totals>` for more information on how to use it on a model. https://openmdao.org/twodocs/versions/latest/features/core_features/working_with_derivatives/approximating_totals.html#cs-guidelines

## Examples

You can check specific combinations of variables by specifying them manually:

In [17]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()

prob.setup()
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [18]:
# manually specify which derivatives to check
prob.check_totals(of=['obj', 'con1'], wrt=['x', 'z'])

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-------------------------------------
Group: SellarDerivatives 'Full Model'
-------------------------------------
  Full Model: 'con1' wrt 'x'
    Analytic Magnitude : 9.806145e-01
          Fd Magnitude : 9.806169e-01 (fd:None)
    Absolute Error (Jan - Jfd) : 2.407248e-06 *

    Relative Error (Jan - Jfd) / Jfd : 2.454831e-06 *

    Raw Analytic Derivative (Jfor)
[[-0.98061448]]

    Raw FD Derivative (Jfd)
[[-0.98061688]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con1' wrt 'z'
    Analytic Magnitude : 9.641989e+00
          Fd Magnitude : 9.641993e+00 (fd:None)
    Absolute Error (Jan - Jfd) : 4.447254e-06 *

    Relative Error (Jan - Jfd) / Jfd : 4.612380e-07

    Raw Analytic Derivative (Jfor)
[[-9.61002186 -0.78449158]]

    Raw FD Derivative (Jfd)
[[-9.61002547 -0.78449417]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  

{('obj', 'x'): {'J_fwd': array([[2.98061391]]),
  'J_fd': array([[2.98061732]]),
  'abs error': ErrorTuple(forward=3.4048181056789417e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=2.9806139134842873, reverse=None, fd=2.980617318302393),
  'rel error': ErrorTuple(forward=1.1423197754276459e-06, reverse=None, forward_reverse=None)},
 ('obj', 'z'): {'J_fwd': array([[9.61001056, 1.78448534]]),
  'J_fd': array([[9.61001417, 1.78448793]]),
  'abs error': ErrorTuple(forward=4.448706699474486e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158459, reverse=None, fd=9.77429125733397),
  'rel error': ErrorTuple(forward=4.5514366027679764e-07, reverse=None, forward_reverse=None)},
 ('con1', 'x'): {'J_fwd': array([[-0.98061448]]),
  'J_fd': array([[-0.98061688]]),
  'abs error': ErrorTuple(forward=2.407248432922948e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=0.9806144751949951, reverse=N

---
Check all the derivatives that the driver will need:

In [19]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [20]:
# check derivatives of all obj+constraints w.r.t all design variables
prob.check_totals()

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-------------------------------------
Group: SellarDerivatives 'Full Model'
-------------------------------------
  Full Model: 'con_cmp1.con1' wrt 'x'
    Analytic Magnitude : 9.806145e-01
          Fd Magnitude : 9.806169e-01 (fd:None)
    Absolute Error (Jan - Jfd) : 2.407248e-06 *

    Relative Error (Jan - Jfd) / Jfd : 2.454831e-06 *

    Raw Analytic Derivative (Jfor)
[[-0.98061448]]

    Raw FD Derivative (Jfd)
[[-0.98061688]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con_cmp1.con1' wrt 'z'
    Analytic Magnitude : 9.641989e+00
          Fd Magnitude : 9.641993e+00 (fd:None)
    Absolute Error (Jan - Jfd) : 4.447254e-06 *

    Relative Error (Jan - Jfd) / Jfd : 4.612380e-07

    Raw Analytic Derivative (Jfor)
[[-9.61002186 -0.78449158]]

    Raw FD Derivative (Jfd)
[[-9.61002547 -0.78449417]]

 - - - - - - - - - - - - - - - - - - - - - - 

{('obj_cmp.obj', 'x'): {'J_fwd': array([[2.98061391]]),
  'J_fd': array([[2.98061732]]),
  'abs error': ErrorTuple(forward=3.4048181056789417e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=2.9806139134842873, reverse=None, fd=2.980617318302393),
  'rel error': ErrorTuple(forward=1.1423197754276459e-06, reverse=None, forward_reverse=None)},
 ('obj_cmp.obj', 'z'): {'J_fwd': array([[9.61001056, 1.78448534]]),
  'J_fd': array([[9.61001417, 1.78448793]]),
  'abs error': ErrorTuple(forward=4.448706699474486e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158459, reverse=None, fd=9.77429125733397),
  'rel error': ErrorTuple(forward=4.5514366027679764e-07, reverse=None, forward_reverse=None)},
 ('con_cmp1.con1', 'x'): {'J_fwd': array([[-0.98061448]]),
  'J_fd': array([[-0.98061688]]),
  'abs error': ErrorTuple(forward=2.407248432922948e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=0.98

---
Use the driver scaled values during the check:

In [21]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()

prob.model.add_design_var('x', lower=-100, upper=100, ref=100.0, ref0=-100.0)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0, ref=3.0)
prob.model.add_constraint('con2', upper=0.0, ref=20.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [22]:
# check derivatives of all driver vars using the declared scaling
prob.check_totals(driver_scaling=True)

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-------------------------------------
Group: SellarDerivatives 'Full Model'
-------------------------------------
  Full Model: 'con_cmp1.con1' wrt 'x'
    Analytic Magnitude : 6.537430e+01
          Fd Magnitude : 6.537446e+01 (fd:None)
    Absolute Error (Jan - Jfd) : 1.604832e-04 *

    Relative Error (Jan - Jfd) / Jfd : 2.454831e-06 *

    Raw Analytic Derivative (Jfor)
[[-65.37429835]]

    Raw FD Derivative (Jfd)
[[-65.37445883]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con_cmp1.con1' wrt 'z'
    Analytic Magnitude : 3.213996e+00
          Fd Magnitude : 3.213998e+00 (fd:None)
    Absolute Error (Jan - Jfd) : 1.482418e-06 *

    Relative Error (Jan - Jfd) / Jfd : 4.612380e-07

    Raw Analytic Derivative (Jfor)
[[-3.20334062 -0.26149719]]

    Raw FD Derivative (Jfd)
[[-3.20334182 -0.26149806]]

 - - - - - - - - - - - - - - - - - - - - - 

{('obj_cmp.obj', 'x'): {'J_fwd': array([[596.1227827]]),
  'J_fd': array([[596.12346366]]),
  'abs error': ErrorTuple(forward=0.0006809636211073666, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=596.1227826968575, reverse=None, fd=596.1234636604786),
  'rel error': ErrorTuple(forward=1.1423197753799683e-06, reverse=None, forward_reverse=None)},
 ('obj_cmp.obj', 'z'): {'J_fwd': array([[9.61001056, 1.78448534]]),
  'J_fd': array([[9.61001417, 1.78448793]]),
  'abs error': ErrorTuple(forward=4.448706699474486e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158459, reverse=None, fd=9.77429125733397),
  'rel error': ErrorTuple(forward=4.5514366027679764e-07, reverse=None, forward_reverse=None)},
 ('con_cmp1.con1', 'x'): {'J_fwd': array([[-65.37429835]]),
  'J_fd': array([[-65.37445883]]),
  'abs error': ErrorTuple(forward=0.00016048322885353627, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=


---
Display the results in a compact format:

In [23]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [24]:
# check derivatives of all obj+constraints w.r.t all design variables
prob.check_totals(compact_print=True)

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-------------------------------------
Group: SellarDerivatives 'Full Model'
-------------------------------------
'<output>'                     wrt '<variable>'                   | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
---------------------------------------------------------------------------------------------------------------------

'con_cmp1.con1'                wrt 'x'                            | 9.8061e-01 | 9.8062e-01 | 2.4072e-06 | 2.4548e-06
'con_cmp1.con1'                wrt 'z'                            | 9.6420e+00 | 9.6420e+00 | 4.4473e-06 | 4.6124e-07
'con_cmp2.con2'                wrt 'x'                            | 9.6928e-02 | 9.6928e-02 | 2.3859e-07 | 2.4615e-06
'con_cmp2.con2'                wrt 'z'                            | 2.2278e+00 | 2.2278e+00 | 3.7386e-07 | 1.6781e-07
'obj_cmp.obj'                  wrt 'x'                            | 2

{('obj_cmp.obj', 'x'): {'J_fwd': array([[2.98061391]]),
  'J_fd': array([[2.98061732]]),
  'abs error': ErrorTuple(forward=3.4048181056789417e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=2.9806139134842873, reverse=None, fd=2.980617318302393),
  'rel error': ErrorTuple(forward=1.1423197754276459e-06, reverse=None, forward_reverse=None)},
 ('obj_cmp.obj', 'z'): {'J_fwd': array([[9.61001056, 1.78448534]]),
  'J_fd': array([[9.61001417, 1.78448793]]),
  'abs error': ErrorTuple(forward=4.448706699474486e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158459, reverse=None, fd=9.77429125733397),
  'rel error': ErrorTuple(forward=4.5514366027679764e-07, reverse=None, forward_reverse=None)},
 ('con_cmp1.con1', 'x'): {'J_fwd': array([[-0.98061448]]),
  'J_fd': array([[-0.98061688]]),
  'abs error': ErrorTuple(forward=2.407248432922948e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=0.98

---
Use complex step instead of finite difference for a more accurate check. We also tighten the tolerance on the nonlinear Gauss-Seidel solver so that we get more accurate converged values.

In [25]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup(force_alloc_complex=True)

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [26]:
prob.model.nonlinear_solver.options['atol'] = 1e-15
prob.model.nonlinear_solver.options['rtol'] = 1e-15

# check derivatives with complex step
prob.check_totals(method='cs')

NL: NLBGS Converged in 7 iterations
NL: NLBGS Converged in 6 iterations
NL: NLBGS Converged in 7 iterations
-------------------------------------
Group: SellarDerivatives 'Full Model'
-------------------------------------
  Full Model: 'con_cmp1.con1' wrt 'x'
    Analytic Magnitude : 9.806145e-01
          Fd Magnitude : 9.806145e-01 (cs:None)
    Absolute Error (Jan - Jfd) : 2.139133e-11

    Relative Error (Jan - Jfd) / Jfd : 2.181421e-11

    Raw Analytic Derivative (Jfor)
[[-0.98061448]]

    Raw FD Derivative (Jfd)
[[-0.98061448]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con_cmp1.con1' wrt 'z'
    Analytic Magnitude : 9.641989e+00
          Fd Magnitude : 9.641989e+00 (cs:None)
    Absolute Error (Jan - Jfd) : 1.806088e-08

    Relative Error (Jan - Jfd) / Jfd : 1.873149e-09

    Raw Analytic Derivative (Jfor)
[[-9.61002186 -0.78449158]]

    Raw FD Derivative (Jfd)
[[-9.61002187 -0.78449158]]

 - - - - - - - - - - - - - - - - - - - - - - - - - 

{('obj_cmp.obj', 'x'): {'J_fwd': array([[2.98061391]]),
  'J_fd': array([[2.98061392]]),
  'abs error': ErrorTuple(forward=9.598640104968581e-09, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=2.9806139134842873, reverse=None, fd=2.9806139230829274),
  'rel error': ErrorTuple(forward=3.220356729408435e-09, reverse=None, forward_reverse=None)},
 ('obj_cmp.obj', 'z'): {'J_fwd': array([[9.61001056, 1.78448534]]),
  'J_fd': array([[9.61001058, 1.78448534]]),
  'abs error': ErrorTuple(forward=1.8061221880559964e-08, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158459, reverse=None, fd=9.774287245923102),
  'rel error': ErrorTuple(forward=1.8478300694603975e-09, reverse=None, forward_reverse=None)},
 ('con_cmp1.con1', 'x'): {'J_fwd': array([[-0.98061448]]),
  'J_fd': array([[-0.98061448]]),
  'abs error': ErrorTuple(forward=2.1391333149267666e-11, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=0.

Note that you need to set the argument `force_alloc_complex` to True when you call `setup`. This makes sure that the vectors allocate space to pass complex numbers. All systems also have an attribute `force_alloc_complex` that can be queried during `setup` in case you need to set up your `Component` or `Group` differently to accomodate complex step.

In [27]:
import openmdao.api as om

class SimpleComp(om.ExplicitComponent):

    def setup(self):
        self.add_input('x', val=1.0)
        self.add_output('y', val=1.0)

        self.declare_partials(of='y', wrt='x')

        if self._force_alloc_complex:
            print("Vectors allocated for complex step.")

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0*inputs['x']

    def compute_partials(self, inputs, partials):
        partials['y', 'x'] = 3.

prob = om.Problem()
prob.model.add_subsystem('comp', SimpleComp())

prob.model.add_design_var('comp.x', lower=-100, upper=100)
prob.model.add_objective('comp.y')

prob.setup(force_alloc_complex=True)

prob.run_model()

prob.check_totals(method='cs')

Vectors allocated for complex step.
-------------------------
Group: Group 'Full Model'
-------------------------
  Full Model: 'comp.y' wrt 'comp.x'
    Analytic Magnitude : 3.000000e+00
          Fd Magnitude : 3.000000e+00 (cs:None)
    Absolute Error (Jan - Jfd) : 0.000000e+00

    Relative Error (Jan - Jfd) / Jfd : 0.000000e+00

    Raw Analytic Derivative (Jfor)
[[3.]]

    Raw FD Derivative (Jfd)
[[3.]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


{('comp.y', 'comp.x'): {'J_fwd': array([[3.]]),
  'J_fd': array([[3.]]),
  'abs error': ErrorTuple(forward=0.0, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=3.0, reverse=None, fd=3.0),
  'rel error': ErrorTuple(forward=0.0, reverse=None, forward_reverse=None)}}


---
Turn off standard output and just view the derivatives in the return:

In [28]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [29]:
# check derivatives of all obj+constraints w.r.t all design variables
totals = prob.check_totals(out_stream=None)

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations


In [30]:
print(totals)

{('obj_cmp.obj', 'x'): {'J_fwd': array([[2.98061391]]), 'J_fd': array([[2.98061732]]), 'abs error': ErrorTuple(forward=3.4048181056789417e-06, reverse=None, forward_reverse=None), 'magnitude': MagnitudeTuple(forward=2.9806139134842873, reverse=None, fd=2.980617318302393), 'rel error': ErrorTuple(forward=1.1423197754276459e-06, reverse=None, forward_reverse=None)}, ('obj_cmp.obj', 'z'): {'J_fwd': array([[9.61001056, 1.78448534]]), 'J_fd': array([[9.61001417, 1.78448793]]), 'abs error': ErrorTuple(forward=4.448706699474486e-06, reverse=None, forward_reverse=None), 'magnitude': MagnitudeTuple(forward=9.774287228158459, reverse=None, fd=9.77429125733397), 'rel error': ErrorTuple(forward=4.5514366027679764e-07, reverse=None, forward_reverse=None)}, ('con_cmp1.con1', 'x'): {'J_fwd': array([[-0.98061448]]), 'J_fd': array([[-0.98061688]]), 'abs error': ErrorTuple(forward=2.407248432922948e-06, reverse=None, forward_reverse=None), 'magnitude': MagnitudeTuple(forward=0.9806144751949951, reverse=